# LLM Application — Summarizing Reviews

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import yaml
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from preprocessing import preprocessed_books_data, preprocessed_books_rating
from llm import summarize_review_t5
from IPython.display import display, HTML

In [3]:
cfg = yaml.safe_load(open("config.yaml"))

In [4]:
books_rating = preprocessed_books_rating(cfg)

In [5]:
books_data = preprocessed_books_data(cfg)

In [6]:
books_rating['full_text_len'] = books_rating['full_text'].str.len()

In [7]:
books_rating.sort_values('full_text_len').tail(10)

,title,user_id,score,full_text,full_text_len
1950860,Torture garden;,AMKC1EJBUXDS2,5.0,"&quot;ART, milady, consists in knowing how to ...",5041
1965928,True History of the Kelly Gang,AG304PU4M3KOB,4.0,A riveting and exciting tale. Peter Carey's Th...,5041
1099102,Overweight Kids,A2Z18D53C95OCH,5.0,Looks at the Physical and the Emotional !. A d...,5041
447478,Dune,A1TWYEFX7HNILJ,5.0,A science-fiction classic!. Dune is commonly c...,5041
130522,Amber Spyglass (10th Anniversary Ed),A2MNL91I56DX98,2.0,Great Adventure but Hypocritical Worldview. Gr...,5041
478745,"Empire of Unreason (The Age of Unreason, Book 3)",A3Q80Q6YFP5W6Q,4.0,Mystic war comes to the Americas. J. Gregory K...,5041
1151749,Prayer Of The Night Shepherd,A18CKXCNMFF35,5.0,Calling All SIr Arthur Conan Doyle fans!. The ...,5041
122983,All He Ever Wanted,A8ANAGMQXVT8R,3.0,All HE ever wanted is right. ** spoiler alert ...,5041
140573,An essay on crimes and punishments,A134C9GVEU5TQE,5.0,Consider Machiavelli Versus Beccaria! A Compar...,5041
1229205,Roots : The Saga of an American Family,AOYTQFRLD8VLP,5.0,Heritage Regained. Alex Haley's monumental tri...,5041


In [8]:
idx = 447478

title = books_rating.loc[idx, 'title']
score = f"{books_rating.loc[idx, 'score']:.1f}"
review = books_rating.loc[idx, 'full_text']

In [9]:
filt_books_data = books_data.query('title == @title')
filt_books_data

,title,description,authors,categories
4562,Dune,Immerse yourself in the world of Denis Villene...,Tanya Lapointe,Art


In [10]:
description = filt_books_data['description'].values[0]
authors = filt_books_data['authors'].drop_duplicates().str.cat(sep=', ')
categories = filt_books_data['categories'].drop_duplicates().str.cat(sep=', ')

In [11]:
summarized_review = summarize_review_t5(
    title=title,
    description=description,
    authors=authors,
    categories=categories,
    score=score,
    review=review,
    model_name='t5-base',
)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [12]:
summarized_review

'Dune is a fascinating story that is a bit of a mystery to me. Dune is a fascinating story. Dune is a great read. Dune Messiah and Children of Dune are both worthwhile reads. Dune Dune Dune Dune Dune Dune. and and and and. and and and. Dune is a great read. Dune Messiah and Dune,'

In [15]:
len(summarized_review)

279

In [16]:
html_content = f"""
<table>
    <tr>
        <td style="vertical-align: top; padding-right: 20px;">{review}</td>
        <td style="vertical-align: top;">{summarized_review}</td>
    </tr>
</table>
"""

display(HTML(html_content))

"Dune is a fascinating story that is a bit of a mystery to me. Dune is a fascinating story. Dune is a great read. Dune Messiah and Children of Dune are both worthwhile reads. Dune Dune Dune Dune Dune Dune. and and and and. and and and. Dune is a great read. Dune Messiah and Dune,"
